# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")
city_data_df['Lat'] = city_data_df['Lat'].astype(float)
city_data_df['Lng'] = city_data_df['Lng'].astype(float)

# Display sample data
city_data_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tobelo,1.7284,128.0095,26.25,82,81,1.53,ID,1699908388
1,1,nola,40.9220,14.5329,18.29,92,80,1.46,IT,1699908389
2,2,cidreira,-30.1811,-50.2056,20.81,96,100,0.87,BR,1699908389
3,3,noumea,-22.2763,166.4572,23.89,99,84,5.73,NC,1699908389
4,4,katsuura,35.1333,140.3000,10.39,52,2,3.78,JP,1699908389


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    color = 'City',
    alpha = 0.5,
    size = "Humidity"
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values

df_ideal_city_conditions = city_data_df[(city_data_df['Max Temp'] > 21) & 
                                        (city_data_df['Max Temp'] < 25) &
                                        (city_data_df['Wind Speed'] < 4) &
                                        (city_data_df['Cloudiness'] < 50) &
                                        (city_data_df['Humidity'] > 40)
                                        ]


# Drop any rows with null values
df_ideal_city_conditions = df_ideal_city_conditions.dropna()

# Display sample data
df_ideal_city_conditions

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
38,38,tolanaro,-25.0319,46.9987,21.27,88,7,2.33,MG,1699908397
175,175,kununurra,-15.7667,128.7333,23.99,100,42,0.00,AU,1699908424
180,180,ha'il,27.5219,41.6907,22.01,46,0,0.00,SA,1699908425
182,182,laguna,38.4210,-121.4238,23.38,48,0,0.45,US,1699908426
183,183,antalaha,-14.9003,50.2788,22.41,88,42,1.55,MG,1699908426
186,186,pisco,-13.7000,-76.2167,21.68,78,0,3.60,PE,1699908426
238,238,rome,34.2570,-85.1647,21.37,44,0,3.09,US,1699908437
253,253,daund,18.4667,74.6000,22.41,51,31,2.90,IN,1699908439
255,255,saint-pierre,-21.3393,55.4781,21.98,68,20,3.60,RE,1699908440
312,312,howrah,22.5892,88.3103,22.00,83,32,2.57,IN,1699908452


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = df_ideal_city_conditions[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
38,tolanaro,MG,-25.0319,46.9987,88,
175,kununurra,AU,-15.7667,128.7333,100,
180,ha'il,SA,27.5219,41.6907,46,
182,laguna,US,38.4210,-121.4238,48,
183,antalaha,MG,-14.9003,50.2788,88,
186,pisco,PE,-13.7000,-76.2167,78,
238,rome,US,34.2570,-85.1647,44,
253,daund,IN,18.4667,74.6000,51,
255,saint-pierre,RE,-21.3393,55.4781,68,
312,howrah,IN,22.5892,88.3103,83,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel
radius = 10000
params = {'categories':'accommodation.hotel',
    "apiKey":geoapify_key,
    "filter": '',
    "bias": ''}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
tolanaro - nearest hotel: Hôtel Mahavokey
kununurra - nearest hotel: Kununurra Country Club Resort
ha'il - nearest hotel: فندق كلاسيك
laguna - nearest hotel: Holiday Inn Express & Suites
antalaha - nearest hotel: Hôtel Florida
pisco - nearest hotel: La Portada
rome - nearest hotel: Days Inn by Wyndham Rome Downtown
daund - nearest hotel: Hotel Tejas
saint-pierre - nearest hotel: Tropic Hotel
howrah - nearest hotel: Sun India Guest House
kiunga - nearest hotel: No hotel found
ula - nearest hotel: No hotel found
ain sukhna - nearest hotel: العين السخنة
ribeira brava - nearest hotel: Cheerfulway Bravamar Hotel
sur - nearest hotel: Sur Hotel
tacoronte - nearest hotel: No hotel found
kone - nearest hotel: Pacifik Appartels
christchurch - nearest hotel: Ibis Hotel Christchurch
urangan - nearest hotel: Ramada
abqaiq - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
38,tolanaro,MG,-25.0319,46.9987,88,Hôtel Mahavokey
175,kununurra,AU,-15.7667,128.7333,100,Kununurra Country Club Resort
180,ha'il,SA,27.5219,41.6907,46,فندق كلاسيك
182,laguna,US,38.4210,-121.4238,48,Holiday Inn Express & Suites
183,antalaha,MG,-14.9003,50.2788,88,Hôtel Florida
186,pisco,PE,-13.7000,-76.2167,78,La Portada
238,rome,US,34.2570,-85.1647,44,Days Inn by Wyndham Rome Downtown
253,daund,IN,18.4667,74.6000,51,Hotel Tejas
255,saint-pierre,RE,-21.3393,55.4781,68,Tropic Hotel
312,howrah,IN,22.5892,88.3103,83,Sun India Guest House


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
%%capture --no-display

# Configure the map plot
map_plot_1 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    color = 'City',
    alpha = 0.5,
    size = 'Humidity',
    hover_cols = ['Hotel Name', 'Country']
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)